In [4]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [5]:
# to provide easy switching between cpu and gpu
CUDA = print(torch.cuda.is_available())
device = torch.device("cuda" if CUDA else "cpu")

False


Part 1 :  **Data Preprocessing**

In [6]:
with open('movie_conversations.txt','r') as conv_file:
  convs = conv_file.readlines()
for conv in convs[:10]:
  print(conv.strip())

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


In [7]:
with open('movie_lines.txt', 'r', errors='replace') as lines_file:
  lines = lines_file.readlines()
for line in lines[:10]:
  print(line.encode('utf-8').strip())

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie."
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?'


In [8]:
line_fields = ['lineID', 'charID', 'movieID', 'char_name', 'text']
lines = {}
with open('movie_lines.txt','r',encoding='iso-8859-1') as file: #encodes ASCII chars, single Byte encoding method
#   head = [next(file) for x in range(10)]
  for line in file:
    vals = line.strip().split('+++$+++')
    lineID = vals[0].strip(' ')
    line_obj = {}
    line_obj['lineID'] = lineID
    for i in range(1,len(vals)):
      line_obj[line_fields[i]] = vals[i]
    lines[line_obj['lineID']] = line_obj

In [9]:
import pprint

In [11]:
conv_fields = ['char1ID', 'char2ID', 'movieID', 'dialoguesIDs']
convs = []
with open('movie_conversations.txt', 'r', encoding='iso-8859-1') as file:
  for line in file:
    vals = line.strip().split('+++$+++')
    conv_obj = {}
    for i in range(len(vals)):
      conv_obj[conv_fields[i]] = vals[i]
    lineIDs = conv_obj['dialoguesIDs'] # get line IDs from dialoguesIDs string, string type
    lineIDs = lineIDs[2:-1] # remove braces from string 
    conv_obj["lines"] = []
    
    for lineID in lineIDs.split(','): # split line IDs separated by ','
      lineID = lineID.strip('\' ')
      try:
        conv_obj["lines"].append(lines[lineID]) #search for that line in lines dict with this linsID
      except KeyError:
        not_found += 1
#         print("here")
        not_found.append(lineID)
    convs.append(conv_obj)
pp = pprint.PrettyPrinter()
pp.pprint(convs[:10])

[{'char1ID': 'u0 ',
  'char2ID': ' u2 ',
  'dialoguesIDs': " ['L194', 'L195', 'L196', 'L197']",
  'lines': [{'charID': ' u0 ',
             'char_name': ' BIANCA ',
             'lineID': 'L194',
             'movieID': ' m0 ',
             'text': ' Can we make this quick?  Roxanne Korrine and Andrew '
                     'Barrett are having an incredibly horrendous public '
                     'break- up on the quad.  Again.'},
            {'charID': ' u2 ',
             'char_name': ' CAMERON ',
             'lineID': 'L195',
             'movieID': ' m0 ',
             'text': " Well, I thought we'd start with pronunciation, if "
                     "that's okay with you."},
            {'charID': ' u0 ',
             'char_name': ' BIANCA ',
             'lineID': 'L196',
             'movieID': ' m0 ',
             'text': ' Not the hacking and gagging and spitting part.  '
                     'Please.'},
            {'charID': ' u2 ',
             'char_name': ' CAMERON ',
 

In [12]:
print(len(convs))

83097


In [13]:
#extract pairs of conversations
conv_pairs = []
for conv in convs :
    for i in range(len(conv['lines'])-1):
        conv1 = conv['lines'][i]['text']
        conv2 = conv['lines'][i+1]['text']
        if conv1 and conv2:
            conv_pairs.append([conv1, conv2])
print(len(conv_pairs))

221282


In [14]:
#create new file containing conversation pairs
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
with open("conv_pairs.txt", "w", encoding='utf-8') as out_file:
    writer = csv.writer(out_file, delimiter=delimiter)
    for pair in conv_pairs:
        writer.writerow(pair)

In [15]:
with open('conv_pairs.txt','rb') as file:
    lines = file.readlines()
    print(len(lines))
    for line in lines[:10]:
        print(line)

221282
b" Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\t Well, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b" Well, I thought we'd start with pronunciation, if that's okay with you.\t Not the hacking and gagging and spitting part.  Please.\r\n"
b" Not the hacking and gagging and spitting part.  Please.\t Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b" You're asking me out.  That's so cute. What's your name again?\t Forget it.\r\n"
b" No, no, it's my fault -- we didn't have a proper introduction ---\t Cameron.\r\n"
b" Cameron.\t The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b" The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\t Seems like she could get a date easy enough...\r\n"
b'

Part 2 : **Processing the words**

In [19]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Vocabulary():
    def __init__(self, name):
        self.name = name # assign name to different vocabularies, if there are > 1
        self.word2ind = {} # dictionary to store unique index for each word in dictionary
        self.word2count = {} # store frequency of each word
        self.ind2word = {PAD_token:"PAD", SOS_token:"SOS", EOS_token:"EOS"}  #id:word
        self.num_words = 3 #PAD, SOS, EOS
    
    def word_count(self, sentence): # function to count word frequency
        for word in sentence.split(' '):
            try:
                self.word2count[word] += 1
            except KeyError:
                self.word2count[word] = 1
    def add_word(self, word):
        try : 
            self.word2count[word] += 1
        except KeyError:
            self.word2ind[word] = self.num_words
            self.word2count[word] = 1
            self.ind2word[num_words] = word
            self.num_words += 1
            
    # keep only those words in vocabulary whose frequency is above certain threshold
    def trim(self, threshold):
#         vocab = []
        for k,v in self.word2count.items():
            if v >= threshold:
                del self.word2count[k]
#                 vocab.append(k)
        # Re-initialise the class variables
#         self.word2ind = {} 
#         self.word2count = {} 
#         self.ind2word = {PAD_token:"PAD", SOS_token:"SOS", EOS_token:"EOS"}  
#         self.num_words = 3 
        
        for word in word2count.keys():
            self.add_word(word)

IndentationError: expected an indented block (<ipython-input-19-e206bdc722a9>, line 33)